In [1]:
# definitions for carrying out bleu score calculations for two outputs
import subprocess
subprocess.run(["pip", "install", "nltk"])
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

def bleu_score(hypothesis, reference):
    from nltk.translate.bleu_score import sentence_bleu
    return sentence_bleu([reference], hypothesis)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
# !pip install rouge-score
subprocess.run(["pip", "install", "rouge-score"])
import rouge_score

from rouge_score import rouge_scorer

def calculate_rouge_score(hypothesis, reference):
   
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores['rougeL'].fmeasure



hypothesis = "The quick brown fox jumps over the lazy dog"
reference = "The quick brown fox jumped over the lazy dog"
rouge_l_fmeasure = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_l_fmeasure}")

hypothesis = "The slow purple fox jumps over the lazy fox"

rouge_Scores = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_Scores}")


ROUGE-L F-measure: 1.0
ROUGE-L F-measure: 0.6666666666666666


In [3]:
# !pip install bert_score
subprocess.run(["pip", "install", "bert_score"]) 
import bert_score

def calculate_bertscore(hypotheses, references, model_type='bert-base-uncased'):
    P, R, F1 = bert_score.score(hypotheses, references, model_type=model_type, lang="en")

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }

# Example usage
hypotheses = ["The quick brown fox jumps over the lazy dog"]
references = ["The quick brown fox jumped over the lazy dog"]
bertscore_result = calculate_bertscore(hypotheses, references)
print(f"BERTScore: {bertscore_result}")

BERTScore: {'precision': 0.9593022465705872, 'recall': 0.9593022465705872, 'f1': 0.9593022465705872}


In [4]:


import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def calculate_perplexity(text, model_name='gpt2'):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        loss = model(input_ids, labels=input_ids)[0]
    return loss.item()

# Example usage
text = "The quick brown fox jumps over the lazy dog"
perplexity = calculate_perplexity(text)
print(f"Perplexity: {perplexity}")

Perplexity: 5.426182746887207


In [5]:
subprocess.run(["pip", "install", "openai"])
import openai
from openai import OpenAI
# !pip install python-dotenv
import os


client = OpenAI(api_key="sk-proj-4Brh6XVArOOG4c7DMUI7T3BlbkFJIZrEaupeqYNS23AIYpnz")

In [6]:
# baseline example

def generate_persona_prompt(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas only for user 2 \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given persona, from which you will have to infer the person of user1 and respond accordingly. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Here are the personas of user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt(text, persona)
# print(f"Response: {response}")


In [7]:
# finetuned example

def generate_persona_prompt_finetuned(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas only for user 2 \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given persona, from which you will have to infer the person of user1 and respond accordingly. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Here are the personas of user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::9VQnLCvi",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt_finetuned(text, persona)
# print(f"Response: {response}")


In [8]:
import pandas as pd

def extract_conversation_parts(csv_filename):
    data = pd.read_csv(csv_filename)
    
    extracted_data = []
    
    for _, row in data.iterrows():
        user1_persona = row['user 1 personas']
        user2_persona = row['user 2 personas']
        conversation = row['Best Generated Conversation']
        
        conversation_lines = conversation.split('\n')
        
        if len(conversation_lines) >= 7:
            context = "\n".join(conversation_lines[:6])
            prompt = conversation_lines[6]
            full_continued_convo = "\n".join(conversation_lines[7:])
            single_continued_convo = conversation_lines[7]
            
            extracted_data.append({
                'user1_persona': user1_persona,
                'user2_persona': user2_persona,
                'context': context,
                'prompt': prompt,
                'finished_convo': full_continued_convo,
                'prompt_response': single_continued_convo
            })
        else:
            print(f"Warning: Conversation is too short in row {row.name}")

    return extracted_data

csv_filename = r'Synthetic-Persona-Chat_valid.csv'
extracted_data = extract_conversation_parts(csv_filename)

trunc_extracted_data = extracted_data[:15]

for data in trunc_extracted_data:
    print("User 1 Persona:", data['user1_persona'])
    print("User 2 Persona:", data['user2_persona'])
    print("Context:", data['context'])
    print("Prompt:", data['prompt'])
    print("Expected response:", data['prompt_response'])
    print()

User 1 Persona: I love to bake cookies.
I have a dogs.
The county wide bake sale is where i feel most at home.
Knitting is my passion.
User 2 Persona: I am a boy.
I can move objects with my mind.
I had to have a transplant.
I was born with my heart outside my body.
Context: User 1: Hi!
User 2: Hello!
User 1: What is your favorite thing to do?
User 2: I like to move objects with my mind.
User 1: That sounds like a lot of fun! I've always wanted to be able to do that.
User 2: It is! It's also really useful.
Prompt: User 1: What else do you like to do?
Expected response: User 2: I like to play video games and read comics.

User 1 Persona: I am an animal activist.
The holidays make me depressed.
I have rainbow hair.
I spend my time bird watching with my cats.
User 2 Persona: I feel old.
I am currently in a juvenile detention center.
I will be released in about a month.
I am here for shoplifting.
Context: User 1: Hi, how are you?
User 2: I'm doing okay. I'm a little bored, but I'm trying to

In [11]:
# baseline validation
def process_validation_data(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:15]  # Taking only the first 15 rows
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses = process_validation_data(csv_filename)

for response in model_responses:
    print("Generated Response:", response)
    print()


Generated Response: User 2: I enjoy exploring my surroundings and testing the limits of my abilities. It's fascinating to see what I can achieve with my mind. Sometimes, I also like to volunteer at the hospital where I had my transplant to give back and support others going through similar experiences. It might sound intense, but it's fulfilling in ways I never imagined.

Generated Response: User 2: Yeah, it's been eye-opening for me. I never really thought about animal rights before, but it's making me see things from a different perspective. Do you have any favorite animals?

Generated Response: User 2: Yeah, there's just something about the intensity and strategy in that battle that really fascinates me. The bravery shown by those in combat is truly remarkable. Do you have any other favorite war documentaries?

Generated Response: That's an exciting field to pursue with a business degree. Investment banking can be challenging but also very rewarding. If you do end up going to Wharto

In [12]:
def process_validation_data_finetuned(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:15]  # Taking only the first 15 rows
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt_finetuned(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses_finetuned = process_validation_data_finetuned(csv_filename)

for response in model_responses_finetuned:
    print("Generated Response:", response)
    print()

Generated Response: User 2: I like to read and write. I also like to play video games.
User 1: I like to read and write too! What kind of books do you like to read?
User 2: I like to read fantasy and science fiction books. I also like to read nonfiction books about history and science.
User 1: I love fantasy and science fiction books too! I've been reading a lot of Brandon Sanderson's books lately.
User 2: I love Brandon Sanderson's books! I've read the Mistborn series and the Stormlight Archive.
User 1: I've read the Mistborn series too! I'm currently reading The Way of Kings.
User 2: The Way of Kings is really good! It's the first book in the Stormlight Archive.
User 1: I'm excited to read the rest of the series. I've heard good things about it.
User 2: You're going to love it! It's one of my favorite fantasy series.
User 1: I'm glad you're enjoying it. What other books do you like to read?
User 2: I also like to read nonfiction books about history and science. I'm really interested 

In [13]:
single_expected_responses = [data['prompt_response'] for data in trunc_extracted_data]
full_expected_responses = [data['finished_convo'] for data in trunc_extracted_data]

single_model_responses = [response.split('\n')[0] for response in model_responses]
single_model_responses

["User 2: I enjoy exploring my surroundings and testing the limits of my abilities. It's fascinating to see what I can achieve with my mind. Sometimes, I also like to volunteer at the hospital where I had my transplant to give back and support others going through similar experiences. It might sound intense, but it's fulfilling in ways I never imagined.",
 "User 2: Yeah, it's been eye-opening for me. I never really thought about animal rights before, but it's making me see things from a different perspective. Do you have any favorite animals?",
 "User 2: Yeah, there's just something about the intensity and strategy in that battle that really fascinates me. The bravery shown by those in combat is truly remarkable. Do you have any other favorite war documentaries?",
 "That's an exciting field to pursue with a business degree. Investment banking can be challenging but also very rewarding. If you do end up going to Wharton or Harvard, you'll definitely have some great opportunities in that

In [14]:
single_model_responses_finetuned = [response.split('\n')[0] for response in model_responses_finetuned]
single_model_responses_finetuned

['User 2: I like to read and write. I also like to play video games.',
 "User 2: Me too! I think animals are so amazing. They're so much more intelligent than we give them credit for.",
 "User 2: Yeah, it's so well-made. What's your favorite part?",
 "User 2: Oh, that's a great field. I have a few friends who work in investment banking and they love it.",
 "User 2: Me too! I've heard they're great performers.",
 "User 2: That's so sweet. I'm sure he's a great companion.",
 "User 2: My name is (name). What's yours?",
 "User 2: Oh, that's cool! What kind of horses do you ride?",
 "User 2: That sounds amazing! I'm so jealous. What's the best thing about being a stunt double?",
 "User 2: I've never listened to them before. What kind of music do they play?",
 "User 2: Roses are beautiful! I've always wanted to have a rose garden, but I don't have the space for it right now.",
 "User 2: You definitely should. It's one of my favorite places in the city.",
 "User 2: Me too! It's a great way to

In [15]:
bertscore_result = calculate_bertscore(single_expected_responses, single_model_responses)
print(f"BERTScore: {bertscore_result}")

BERTScore: {'precision': 0.6122077107429504, 'recall': 0.4830414056777954, 'f1': 0.5368104577064514}


In [16]:
bertscore_result_finetuned = calculate_bertscore(single_expected_responses, single_model_responses_finetuned)
print(f"BERTScore: {bertscore_result_finetuned}")

BERTScore: {'precision': 0.7813934683799744, 'recall': 0.7059955596923828, 'f1': 0.737503707408905}
